# Imports

In [2]:
import tensorflow as tf
import numpy as np
import math
import pydot
import os
#import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import plot_model
from contextlib import redirect_stdout
from tensorflow.keras.utils import to_categorical

## Inception Module

In [19]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,filters_5x5_reduce,filters_5x5,filters_max_pool,name=None):
    """ Inception Module """
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    max_pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    max_pool = Conv2D(filters_max_pool, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(max_pool)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, max_pool], axis=3, name=name)
    
    return output

### We will then create the GoogLeNet architecture, as mentioned in the paper.

In [20]:
kernel_init = tf.keras.initializers.glorot_uniform()
bias_init = tf.keras.initializers.Constant(value=0.2)

### GoogleNet
#### inception_v1

In [21]:
def GoogLeNet(input_shape = None):
    if input_shape:
        input_layer = Input(shape=input_shape)
        
        # Convolution operation.
        x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
        x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
        x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
        x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
        x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
        
        # First Inception block
        x = inception_module(x, filters_1x1=64, filters_3x3_reduce=96,filters_3x3=128,
                             filters_5x5_reduce=16, filters_5x5=32, filters_max_pool=32,
                             name='inception_3a')
        # Second Inception block
        x = inception_module(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192,
                             filters_5x5_reduce=32, filters_5x5=96, filters_max_pool=64,
                             name='inception_3b')

        x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
        
        # Third Inception block
        x = inception_module(x, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208,
                             filters_5x5_reduce=16, filters_5x5=48, filters_max_pool=64,
                             name='inception_4a')

        # First Ouput layer
        output_1 = AveragePooling2D((5, 5), strides=3)(x)
        output_1 = Conv2D(128, (1, 1), padding='same', activation='relu')(output_1)
        output_1 = Flatten()(output_1)
        output_1 = Dense(1024, activation='relu')(output_1)
        output_1 = Dropout(0.7)(output_1)
        output_1 = Dense(10, activation='softmax', name='auxilliary_output_1')(output_1)
        
        # Fourth Inception block
        x = inception_module(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224,
                             filters_5x5_reduce=24, filters_5x5=64, filters_max_pool=64,
                             name='inception_4b')
        
        # Fifth Inception block
        x = inception_module(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256,
                             filters_5x5_reduce=24, filters_5x5=64, filters_max_pool=64,
                             name='inception_4c')
        
        # Sixth Inception block
        x = inception_module(x, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288,
                             filters_5x5_reduce=32, filters_5x5=64, filters_max_pool=64,
                             name='inception_4d')

        # Second output layer.
        output_2 = AveragePooling2D((5, 5), strides=3)(x)
        output_2 = Conv2D(128, (1, 1), padding='same', activation='relu')(output_2)
        output_2 = Flatten()(output_2)
        output_2 = Dense(1024, activation='relu')(output_2)
        output_2 = Dropout(0.7)(output_2)
        output_2 = Dense(10, activation='softmax', name='auxilliary_output_2')(output_2)
        
        # Seventh Inception block
        x = inception_module(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320,
                             filters_5x5_reduce=32, filters_5x5=128, filters_max_pool=128,
                             name='inception_4e')

        x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
        
        # Eighth Inception block
        x = inception_module(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320,
                             filters_5x5_reduce=32, filters_5x5=128, filters_max_pool=128,
                             name='inception_5a')
        
        # Ninth Inception block
        x = inception_module(x, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384,
                             filters_5x5_reduce=48, filters_5x5=128, filters_max_pool=128,
                             name='inception_5b')

        x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

        x = Dropout(0.4)(x)
        
        # Final output layer
        output = Dense(10, activation='softmax', name='output')(x)
        
        model = Model(input_layer, [output, output_1, output_2], name='inception_v1')
        
        return model
        

In [22]:
inception = GoogLeNet(input_shape = (224, 224, 3))

In [11]:
def summary(output=None, target=None, model = None):
        """ Show / Save model structure (summary) """
        if model is not None:
            model.summary()
            if target is not None:
                os.makedirs(output, exist_ok=True)
                with open(os.path.join(output, target), "w") as f:
                    with redirect_stdout(f):
                        model.summary()

In [13]:
summary(output= './summary', target='summary', model = inception)

Model: "inception_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
_______________________________________________________________________________________

In [14]:
def model_plot(model = None, file_name=None):
    if model and file_name is not None:
        print('Model plotting...')
        
        plot_model(model, to_file = file_name, show_shapes=True,
                   show_layer_names=True,
                   rankdir='LB', expand_nested=True, dpi=96)
        print('Model plotted..')

In [16]:
model_plot(model = inception, file_name='Inception_model.png')

Model plotting...
Model plotted..


# Training

In [24]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

inception.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [24]:
#history = inception.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])